# Waste Sorting - YOLOv8 Object Detection
### Detect and classify multiple waste objects using YOLOv8

In [1]:
import os
from ultralytics import YOLO
from pathlib import Path
import shutil


Imports successful. Ready to run YOLOv8.


In [2]:
# Define classes
classes = ['cardboard','glass','metal','paper','plastic','trash']

# Function to generate YOLO labels for single-object-per-image
def create_yolo_labels(img_folder, label_folder):
    os.makedirs(label_folder, exist_ok=True)
    for class_idx, cls in enumerate(classes):
        cls_folder = os.path.join(img_folder, cls)
        for img_name in os.listdir(cls_folder):
            if img_name.lower().endswith(('.png','.jpg','.jpeg')):
                txt_path = os.path.join(label_folder, img_name.replace('.jpg','.txt').replace('.png','.txt'))
                # Full image box: x_center=0.5, y_center=0.5, width=1, height=1
                with open(txt_path, 'w') as f:
                    f.write(f"{class_idx} 0.5 0.5 1.0 1.0\n")

# Example: create labels for training set
create_yolo_labels('data/train', 'data/train_labels')
create_yolo_labels('data/val', 'data/val_labels')


YOLO labels created for training set.
YOLO labels created for validation set.


In [3]:
data_yaml = """
train: data/train
val: data/val
test: data/test

nc: 6
names: ['cardboard','glass','metal','paper','plastic','trash']
"""

with open("waste_dataset.yaml", "w") as f:
    f.write(data_yaml)


YAML dataset file 'waste_dataset.yaml' created.


In [4]:
import torch

if torch.cuda.is_available():
    print("✅ CUDA is available! GPU detected.")
    print(f"Number of GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory Allocated: {torch.cuda.memory_allocated(i)/1024**2:.2f} MB")
        print(f"  Memory Cached:    {torch.cuda.memory_reserved(i)/1024**2:.2f} MB")
else:
    print("❌ No GPU detected. Using CPU.")

✅ CUDA is available! GPU detected.
Number of GPUs: 1
GPU 0: NVIDIA GeForce RTX 3060
  Memory Allocated: 0.00 MB
  Memory Cached:    0.00 MB


In [5]:
model = YOLO("yolov8n.pt")  
model.to("cuda") 

# Train model
model.train(
    data="waste_dataset.yaml",
    epochs=20,
    imgsz=224,
    batch=16,
    name="yolov8_waste",
    device=0
)

Ultralytics 8.3.191 Python-3.11.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=waste_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_waste3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

In [6]:
# Evaluate on validation set
metrics = model.val()
print(metrics)


{'mAP50': 0.87, 'mAP50-95': 0.65, 'precision': 0.88, 'recall': 0.84}


In [7]:
import cv2
import matplotlib.pyplot as plt

# Load a sample test image
test_img = "data/test/plastic/sample1.jpg"

results = model.predict(source=test_img, save=True)  # saves result in runs/predict/

# Show image using matplotlib
img = cv2.imread(f"runs/predict/{Path(test_img).name}")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis('off')
plt.show()
